# Mocap Data Pre-processing

In [1411]:
using LinearAlgebra, Random
using StatsBase, Statistics
using Distributions, MultivariateStats   # Categorical, PCA
using Quaternions    # For manipulating 3D Geometry
using MeshCat        # For web visualisation / animation
using PyPlot         # Plotting
using AxUtil, Flux   # Optimisation

# small utils libraries
using ProgressMeter, Formatting, ArgCheck
using DelimitedFiles, NPZ, BSON

In [2]:
DIR_MOCAP_MTDS = "." #"""../../../mocap-mtds/";   # different cos I'm in dev folder

# Data loading and transformation utils
include(joinpath(DIR_MOCAP_MTDS, "io.jl"))

# MeshCat skeleton visualisation tools
include(joinpath(DIR_MOCAP_MTDS, "mocap_viz.jl"))

In [3]:
cmu_loco = readdlm("../data/mocap/cmu/cmu_locomotion_lkp.txt", '\t')[:,1];
database = "../data/mocap/holden/cmu"
files_cmu = [joinpath(database, f * ".bvh") for f in cmu_loco]
files_cmu = collect(filter(x-> isfile(x) && x !== "rest.bvh", files_cmu));

database = "../data/mocap/edin-style-transfer/"
files_edin = [joinpath(database, f) for f in readdir(database)];

### Read in example data

In [191]:
proc = mocapio.process_file(files_cmu[40]);   
# 54 = PACING/STOP, 40 = BACKWARD, 115=TRIUMPHANT, 190=TWISTY, not 198 (JUMP)!

In [347]:
include(joinpath(DIR_MOCAP_MTDS, "io.jl"))

In [571]:
fps = 60

In [1413]:
proc = mocapio.process_file(files_edin[2], fps=fps);   

In [1414]:
yy = mocapio.construct_outputs(proc, fps=fps);

In [1415]:
u = mocapio.construct_inputs(proc; joint_pos=true, fps=fps, direction=:relative)

In [1416]:
ii = 1

In [1431]:
@showprogress for (_i, ii) in enumerate(range(1,stop=size(u,1), step=2))
    PyPlot.cla()
    tmp_new = hcat(u[ii,1:12], u[ii,13:24])
    plot(tmp_new[:,1], tmp_new[:,2])
    scatter(0,0)
    # arrow(0,0, diff(tmp_new[7:8,1])[1], diff(tmp_new[7:8,2])[1], width=.2)
    for j in 1:11
        dirΔ = [tmp_new[j+1,1] - tmp_new[j,1], tmp_new[j+1,2] - tmp_new[j,2]]
        dirΔ /= sqrt(sum(z->z^2, dirΔ))
        std_cθ, std_sθ = add_angle(u[ii,24+j], u[ii,36+j], 0, -1)
        cθ, sθ = add_angle(dirΔ[1], dirΔ[2], std_cθ, -std_sθ)
        @assert isapprox(reim(complex(0, 1) * complex(u[ii,24+j], -u[ii,36+j]) * complex(dirΔ[1], dirΔ[2])) |> collect, 
            [cθ, sθ])
        arrow(tmp_new[j,1],tmp_new[j,2], cθ, sθ, width=.05)
    end

#     _tmp_dir = reim(-complex(1, 0) *complex(u[ii,24+8], -u[ii,36+8]) )  # u[34+ii,8], u[34+ii,12+8]
#     @assert isapprox(collect(_tmp_dir), [-u[ii,24+8], u[ii,36+8]])
#     arrow(0, 0, _tmp_dir[1],_tmp_dir[2], width=0.2/sqrt(sum(diff([gca().get_xlim()...]))), 
#     color=ColorMap("tab10")(1))
#     plot(zeros(2), [-u[ii,36+8], u[ii,36+8]]*1.5, color="k", linewidth=0.4)

    gca().set_xlim(-20, 20)
    # gca().set_ylim(-15, 15)
    gca().set_aspect("equal")
    savefig(format("img/traj_{:03d}.png", _i))
end

In [ ]:
extract_reim(x)

In [1213]:
complex(0,1.0)

In [1151]:
plot(atan.(u[ii-50:ii+50, 24+7], u[ii-50:ii+50, 36+7]))
gcf().set_size_inches(3,2)

## Getting ready for training

In [1410]:
# Load data from raw BVH files (1-2 mins), or load a saved version
DATA_FROM_SAVED = false
fps = 30

if DATA_FROM_SAVED
    Xs = BSON.load("edin_Xs.bson")[:Xs];
    Ys = BSON.load("edin_Ys.bson")[:Ys];
else
    proc_files = map(files_edin) do f
        mocapio.process_file(f, fps=fps);
    end;
    println("loaded files..."); flush(stdout)
    Xs = map(proc_files) do proc
        convert(Matrix{Float32}, mocapio.construct_inputs(proc; joint_pos=true, fps=fps))
    end;
    println("Xs finished processing..."); flush(stdout)
    Ys = map(proc_files) do proc
        convert(Matrix{Float32}, 
            mocapio.construct_outputs(proc; include_ftcontact=false, fps=fps))
    end;
    println("Ys finished processing..."); 
    
    println("Do you want to save? (y/n)")
    
    for i = 1:10
        userinput = uppercase(chomp(readline()))[1]
        if userinput == 'Y'
            println("SAVING..."); flush(stdout)
            BSON.bson("edin_Xs_30fps.bson", Xs=Xs)
            BSON.bson("edin_Ys_30fps.bson", Ys=Ys)
            break
        elseif userinput == 'N'
            break
        end
    end
end;

Ysraw = Ys
Xsraw = Xs;

In [127]:
if !(@isdefined vis) 
    # Create a new visualizer instance (MeshCat.jl)
    vis = Visualizer()
    open(vis)
end
vis = mocapviz.create_animation([mocapio.reconstruct_modelled(Ysraw[1])[1:200,:,:]], 
    "test"; vis=vis, linemesh=mocapviz.yellowmesh, camera=:back)

### Need to standardize inputs and outputs

`MLPreprocessing.jl` contains something like a port of `StandardScaler` from `sklearn`. However, as of 18/05/2019 it appears to be essentially broken, and given its broad generality to lots of array types, I don't want to get into this whole thing. So I'm just going to do define something myself.

In [9]:
?MyStandardScaler

In [5]:
include(joinpath(DIR_MOCAP_MTDS, "util.jl"))
import .mocaputil: MyStandardScaler, scale_transform
# const scale_transform = mocaputil.transform

In [6]:
standardize_Y = fit(MyStandardScaler, reduce(vcat, Ysraw),  1)
standardize_X = fit(MyStandardScaler, reduce(vcat, Xsraw),  1)

Ys = [scale_transform(standardize_Y, y) for y in Ysraw];
Xs = [scale_transform(standardize_X, x) for x in Xsraw];

note that we can reconstruct the original data via the command:

    invert(standardize_Y, y)
    invert(standardize_X, x)
    
in the relevant array comprehensions.

### It is worth checking to see if it makes sense to scale everything
As shown in the input/output columns below, there is nothing that appears problematic in doing this

In [12]:
?mocapio.construct_inputs

In [10]:
?mocapio.construct_outputs

In [140]:
Ys[1][:,65:end]

In [167]:
?mocaputil.MyStandardScalar

In [159]:
mutable struct OutputDifferencer{T}
    first_frame::Array{T, 1}
    operate_on::AbstractArray{L where L <: Int,1}
end

Base.length(s::OutputDifferencer) = length(s.first_frame, 1)
Base.copy(s::OutputDifferencer) = OutputDifferencer(copy(s.first_frame), copy(s.operate_on))

function StatsBase.fit(::Type{OutputDifferencer}, Y)
    @argcheck size(Y, 2) in [64, 68]
    return OutputDifferencer(Y[1,:], 4:64)
end

function difference_transform(s::OutputDifferencer, Y)
    @assert Y[1,:] == s.first_frame
    return hcat(Y[2:end,1:(s.operate_on[1] - 1)], 
        diff(Y[:,s.operate_on], dims=1),
        Y[2:end,(s.operate_on[end] + 1):end])
end

function fit_transform(::Type{OutputDifferencer}, Y)
    s = fit(OutputDifferencer, Y)
    return s, difference_transform(s, Y)
end

function invert(s::OutputDifferencer, Y)
    tr_first = reshape(s.first_frame, 1, :)
    inv = cumsum(vcat(tr_first[:,s.operate_on], Y[:,s.operate_on]), dims=1)
    return vcat(tr_first,
                hcat(Y[:,1:(s.operate_on[1] - 1)], 
                    inv[2:end,:],
                    Y[:,(s.operate_on[end] + 1):end])
                )
end

In [144]:
s, Ydiff = fit_transform(OutputDifferencer, Ys[1])

In [149]:
imshow(Ys[1][1:200,:]); gca().set_aspect("auto")

In [166]:
all(isapprox.(invert(s, Ydiff), Ys[1], atol=1e-5))

## Last checks with visualisation

In order to visualise the *direction* of ±60 frames, we have to essentially perform FK since these directions are all relative to the forward direction at the current location. I've done this using fairly vanilla trigonometric functions below as my knowledge of quaternions and manipulating 3D graphics is not as good as it could be.

In [161]:
function angle_to_z_axis(v)
    cθ, sθ = mocapio._trigvecs(reshape(v, 1, 2), reshape([0f0 1f0], 1, 2))
    atan(-sθ[1], cθ[1])
end
rotationm(θ) = [cos(θ) sin(θ); -sin(θ) cos(θ)]

function add_angle(c1, s1, c2, s2)
    θ₁, θ₂ = atan(s1,c1), atan(s2, c2)
    return cos(θ₁ + θ₂), sin(θ₁ + θ₂)
end

In [1203]:
Complex(-1/sqrt(2), -1/sqrt(2)) * Complex(+1/sqrt(2), -1/sqrt(2)+0.05)

In [ ]:
atan()

In [1190]:
methods(angle)

In [ ]:
mutable struct MyAngle
    

In [114]:
ii = 436

In [112]:
plot(atan.(Xsraw[4][ii-50:ii+50, 24+7], Xsraw[4][ii-50:ii+50, 36+7]))
gcf().set_size_inches(3,2)

In [1205]:
angle(Complex(0, -1))

In [116]:
tmp_new = hcat(Xsraw[4][ii,1:12], Xsraw[4][ii,13:24])
plot(tmp_new[:,1], tmp_new[:,2])
scatter(0,0)
arrow(0,0, diff(tmp_new[7:8,1])[1], diff(tmp_new[7:8,2])[1], width=.2)
for j in 1:11
    dirΔ = [tmp_new[j+1,1] - tmp_new[j,1], tmp_new[j+1,2] - tmp_new[j,2]]
    dirΔ /= sqrt(sum(z->z^2, dirΔ))
    cθ, sθ = Complex(0, -1) * Complex(Xsraw[4][ii,24+j], Xsraw[4][ii,36+j]) * Complex(dirΔ[1], dirΔ[2])
    
#     std_cθ, std_sθ = add_angle(Xsraw[4][ii,24+j], Xsraw[4][ii,36+j], 0, -1)
#     cθ, sθ = add_angle(dirΔ[1], dirΔ[2], std_cθ, -std_sθ)
    arrow(tmp_new[j,1],tmp_new[j,2], cθ, sθ, width=.05)
end
gca().set_aspect("equal")
# ii += 5

In [108]:
if !(@isdefined vis) 
    # Create a new visualizer instance (MeshCat.jl)
    vis = Visualizer()
    open(vis)
end
vis = mocapviz.create_animation([mocapio.reconstruct_modelled(Ysraw[4])[300:550,:,:]], 
    "test"; vis=vis, linemesh=mocapviz.yellowmesh, camera=:back)

## Everything looks good

**Note** the input traces look slightly odd during sharp/stopping "about" turns as frequently the trajectory turns in the opposite direction to the body. Since the turning circle is so tight, it is probably not well defined which way the trajectory turns, and anchoring to the *actual position* of the root joint is a good a way as any. The point is that while it may look a little odd, the processing is doing the right thing, and the information will be available to the model.

## Look at principal components

In [218]:
?mocapio.construct_inputs

In [121]:
allE = reduce(vcat, Ys);
allE = convert(Matrix{Float32}, allE);

In [124]:
zsc(x, dims) = (x .- mean(x, dims=dims)) ./ std(x, dims=dims)

pc_all = fit(PCA, zsc(allE[:,4:63], 1)', pratio=0.999)

varexpl = cumsum(principalvars(pc_all))/tvar(pc_all)
bd=findfirst.([varexpl .> x for x in [0.9,0.95,0.99]])
plot(1:length(varexpl), varexpl)
gca().axhline(1, linestyle=":")
for b in bd
    plot([b,b], [varexpl[1], varexpl[b]], color=ColorMap("tab10")(7), linestyle=":")
    plot([.5, b], [varexpl[b], varexpl[b]], color=ColorMap("tab10")(7), linestyle=":")
end
gca().set_xlim(0.5,25.5); gca().set_ylim(varexpl[1],1.025);
gcf().set_size_inches(3,2)

#### Note
This is different (worse, i.e. need more PCs) than the graph I showed to Chris. This is because I was in the Eulerian frame, where direction(s) of movement probably captured most of PCs. Removing these high variance directions, common to all joints makes it much harder.